# COMP47590: Advanced Machine Learning
# Assignment 1: Implementing Perceptrons

- Student 1 Name: Carl Fabian Winkler
- Student 1 Number: 20207528
- Student 2 Name: David Moreno Boras
- Student 2 Number: 21200646

## Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split

"""
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.utils.validation import check_X_y, check_array, check_is_fitted, check_random_state
from sklearn.utils.multiclass import unique_labels
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.utils import resample"""

'\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.model_selection import cross_val_score\nfrom sklearn.model_selection import train_test_split\n\nfrom sklearn.utils.validation import check_X_y, check_array, check_is_fitted, check_random_state\nfrom sklearn.utils.multiclass import unique_labels\nfrom sklearn import preprocessing\nfrom sklearn import metrics\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.model_selection import cross_val_score\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.utils import resample'

## Task 1: The Perceptron Classifier

Define the PerceptronClassifier class

In [2]:
class Layer():
    def __init__(self, n_in, n_out, activation = 'Sigmoid', init = 'Xavier'):
        self.activation = activation
        # XW + b = y ; We input more than one sample per pass...
        
        self.weights = np.zeros((n_in, n_out))
        self.biases = np.zeros((n_out))
        if init == 'Xavier':
            var = np.sqrt(6.0 / (n_in + n_out))
            for i in range(n_in):
                for j in range(n_out):
                      self.weights[i,j] = np.float32(np.random.uniform(-var, var))
        
        self.d_w = np.zeros(weights.shape)
        self.d_b = np.zeros(biases.shape)
        #print("Weights:", self.weights.shape)
        #print(self.weights) 
        
    def getWeights(self):
        return self.weights

    def forward(self, x):
        """print("X:", x.shape)
        print(x)
        print("Weights:", self.weights.shape)
        print(self.weights)"""
        z = x @ self.weights + self.biases
        #print("Weights:", self.weights.shape)
        if self.activation == 'Sigmoid':
            out = 1 / (1 + np.exp(-z))
        elif self.activation == 'ReLu':
            out = np.maximum(z, 0)
        elif self.activation == 'TanH':
            out = np.tanh(z)
        else:
            out = z
        
        self.cache = (x, z)
        
        return out    
    
    def backward(self, d_out):
        inputs, z = self.cache
        weight = self.weights
        bias = self.biases
        #print(self.weights)
        if self.activation == 'Sigmoid':
            d_act = d_out * (1 / (1 + np.exp(-z))) * (1 - 1 / (1 + np.exp(-z)))
        elif self.activation == 'ReLu':
            d_act = d_out * (z > 0)
            
        elif self.activation == 'TanH':
            d_act = d_out * np.square(z)
        else:
            d_act = z
            
        d_inputs = d_act @ weight.T
        self.d_w = inputs.T @ d_act
        self.d_b = d_act.sum(axis=0) 
        
        return d_inputs#, self.d_w, self.d_b
    
    def update_gd_params(self, lr):
        #print("Weight upd: ", self.d_w)
        self.weights = self.weights - lr * self.d_w
        self.biases = self.biases - lr * self.d_b

class PerceptronClassifier(BaseEstimator, ClassifierMixin, ):
    
    """
    Parameters
    ----------
    Attributes
    ----------
    Notes
    -----
    See also
    --------
    Examples
    --------
    """
    # Constructor for the classifier object
    def __init__(self, in_dim, out_dim, hidden_units, n_layers, activation = 'Sigmoid', 
                 learning_rate = 0.01, weight_decay = 0, epochs = -1, regularisation = 'L2'):

        """Setup a Perceptron classifier .
        Parameters
        ----------
        Returns
        -------
        """     
        
        self.layers = []
        self.lr = learning_rate
        self.regularisation = regularisation
        
        self.layers.append(Layer(in_dim, hidden_units, activation, 'Xavier'))
        for l in range(n_layers):
            self.layers.append(Layer(hidden_units, hidden_units, activation, 'Xavier'))
        self.layers.append(Layer(hidden_units, out_dim, activation, 'Xavier'))
        
        print("Layers:", len(self.layers))
        
        # Initialise class variabels
    def forward(self, X):
        out = self.layers[0].forward(X)
        for layer in self.layers[1:]:
            out = layer.forward(out)
        return out
                
    def backward(self, in_grad):
        i = len(self.layers) - 2 
        # d_inputs, _, _ = lay.backward(in_grad)
        next_grad = self.layers[i+1].backward(in_grad)
        while i >= 0:
            next_grad = self.layers[i].backward(next_grad)
            i -= 1
        
    def l2_loss(self, y_hat, pred):
        # totalSum = 0
        # for layer in self.layers:
        #     totalSum = totalSum + np.sum(np.sum(layer.getWeights())
        return -y_hat-np.squeeze(pred)

    def loss(self, y_hat, pred):
        return -np.expand_dims(y_hat-np.squeeze(pred),axis=1)
        
    # The fit function to train a classifier
    def fit(self, X, y, epochs = 30):
        # WRITE CODE HERE
        for i in range(epochs):
            out = self.forward(X)
            print("Prediction:",out)
            
            if (self.regularisation == 'L2'):
                grad = self.l2_loss(y, out)
            else:
                grad = self.loss(y, out)
            
            # Backpropagation
            self.backward(grad)
            
            # Update weights and biases
            for layer in self.layers:
                layer.update_gd_params(self.lr)
        return
    
    # The predict function to make a set of predictions for a set of query instances
    def predict(self, X):
        return self.forward(X)
    
    # The predict_proba function to make a set of predictions for a set of query instances. This returns a set of class distributions.
    def predict_proba(self, X):
        tmp = self.forward(X)
        sum1 = tmp.sum(axis = 1)
        out = X.T / sum1
        out = out.T
        return out

## Task 2: Evaluation

Load the Diabethic Retinopathy dataset

In [3]:
x = np.array([[1,1],[2,2]])


D = 2
N = 2
H = 1


weights = np.ones((2,1))
biases = np.ones((1))  
              


In [4]:
#(self, in_dim, out_dim, hidden_units, n_layers,
x = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([0,1,1,0])
clf = PerceptronClassifier(2, 1, 5, 0, regularisation='None', learning_rate = 3,activation='Sigmoid')
clf.fit(x, y,epochs=1000)

Layers: 2
Prediction: [[0.35530089]
 [0.32849397]
 [0.431499  ]
 [0.40294111]]
Prediction: [[0.54971083]
 [0.54617014]
 [0.62369592]
 [0.62003732]]
Prediction: [[0.407522  ]
 [0.38737046]
 [0.48211755]
 [0.46079955]]
Prediction: [[0.51962821]
 [0.51380679]
 [0.59142486]
 [0.5847687 ]]
Prediction: [[0.42968185]
 [0.41340076]
 [0.50098246]
 [0.48303679]]
Prediction: [[0.50526368]
 [0.49898205]
 [0.57427656]
 [0.56630094]]
Prediction: [[0.44234346]
 [0.4287596 ]
 [0.51058697]
 [0.49464542]]
Prediction: [[0.49722133]
 [0.4910979 ]
 [0.56366525]
 [0.55500682]]
Prediction: [[0.45030243]
 [0.43873504]
 [0.5159542 ]
 [0.50129138]]
Prediction: [[0.49239675]
 [0.48669073]
 [0.55665443]
 [0.5476026 ]]
Prediction: [[0.45553999]
 [0.44555178]
 [0.51908133]
 [0.50524768]]
Prediction: [[0.48938655]
 [0.48421622]
 [0.55187136]
 [0.54256055]]
Prediction: [[0.45905055]
 [0.45034566]
 [0.52093667]
 [0.5076238 ]]
Prediction: [[0.48746426]
 [0.48288823]
 [0.54858273]
 [0.53906835]]
Prediction: [[0.46138195

In [5]:

x = np.array([[0.74552196],
 [0.76334402]])
y = np.ones(2)

In [6]:
np.expand_dims(np.squeeze(x)-y,axis=1)

array([[-0.25447804],
       [-0.23665598]])

Evaluate the perfomrance of the perceptron classifier on the daibetic retinopathy dataset.

In [7]:
x = np.array([1,1])
clf.predict(x)


array([0.03007119])

In [8]:
plt.plot(outpur)

NameError: name 'outpur' is not defined

## Task 3 & 4: Add Different Activations & Regularisation

Reimplement the PerceptronClassifier class adding an activation function option and L2 regularisation. 

In [ ]:
class PerceptronClassifier2(BaseEstimator, ClassifierMixin):
    """
    """
    # Constructor for the classifier object
    def __init__(self):

        """Setup a Perceptron classifier .
        Parameters
        ----------

        
        Returns
        -------

        """     

        # Initialise ranomd state if set
        self.random_state = random_state
        
        # Initialise class variabels

        
    # The fit function to train a classifier
    def fit(self, X, y):
        
        # WRITE CODE HERE
        

    # The predict function to make a set of predictions for a set of query instances
    def predict(self, X):

        # WRITE CODE HERE
    
    # The predict_proba function to make a set of predictions for a set of query instances. This returns a set of class distributions.
    def predict_proba(self, X):
        
        # WRITE CODE HERE

Load the dataset and explore it.

## Task 5: Reflect on the Performance of the Different Models Evaluated

Perform hyper-parameter tuning and evaluate models. 

## Test the Diabetic Retiniphaty dataset

In [10]:
diabetic_af = pd.read_csv('messidor_features.csv', na_values='?', index_col = 0)
diabetic_af.head()
y = diabetic_af.pop('Class').values
x_raw = diabetic_af.values
print("Features: ", x_raw[0:2])
print("Class: ", y[0:10])

Features:  [[1.0000000e+00 2.2000000e+01 2.2000000e+01 2.2000000e+01 1.9000000e+01
  1.8000000e+01 1.4000000e+01 4.9895756e+01 1.7775994e+01 5.2709200e+00
  7.7176100e-01 1.8632000e-02 6.8640000e-03 3.9230000e-03 3.9230000e-03
  4.8690300e-01 1.0002500e-01 1.0000000e+00]
 [1.0000000e+00 2.4000000e+01 2.4000000e+01 2.2000000e+01 1.8000000e+01
  1.6000000e+01 1.3000000e+01 5.7709936e+01 2.3799994e+01 3.3254230e+00
  2.3418500e-01 3.9030000e-03 3.9030000e-03 3.9030000e-03 3.9030000e-03
  5.2090800e-01 1.4441400e-01 0.0000000e+00]]
Class:  ["b'0'" "b'0'" "b'1'" "b'0'" "b'1'" "b'1'" "b'1'" "b'0'" "b'1'" "b'1'"]


In [11]:
print(y)
y_numbers = list([int(x[2]) for x in y])
y_numbers = np.array(y_numbers)
print(y_numbers)

["b'0'" "b'0'" "b'1'" ... "b'0'" "b'1'" "b'0'"]
[0 0 1 ... 0 1 0]


### Train and predict using our classifier

With a single split

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x_raw, y_numbers, shuffle=True, train_size = 0.01)
clf = PerceptronClassifier(len(x_train[0]), 1, 10, 3)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
accuracy = metrics.accuracy_score(y_pred, y_test)

Layers: 5
Prediction: [[0.37160991]
 [0.37073331]
 [0.37080128]
 [0.37079213]
 [0.37079557]
 [0.37071694]
 [0.37079727]
 [0.37070617]
 [0.37079573]
 [0.37104706]
 [0.37072107]]


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 11)

### Grid search

Do grid search with the train set, use the test set for evaluation

In [ ]:
cv_folds = 5
param_grid ={'activation': ['Sigmoid', 'ReLu', 'TanH'], 'regularisation':['None', 'L2']}

# Perform the search
tuned_perceptron = GridSearchCV(PerceptronClassifier(), \
                            param_grid, cv=cv_folds, verbose = 2, \
                            n_jobs = -1)
cross_val_score(clf, x_train, y_train, cv=10)